In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

path = "../../dane/8CPU_20RAM/3600s/all_merged.csv"

In [2]:
# Załaduj dane
df = pd.read_csv(path)

In [3]:
for col in ['replicaId', 'endpointUrl_methods']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [4]:
# Wybór cech
features = [
            # 'timestamp',
            # 'apiTime_methods',
            # 'applicationTime_methods',
            # 'databaseTime_methods',
            'endpointUrl_methods',
            'queueSizeForward_methods',
            'queueSizeBack_methods',
            'cpuUsage_stock',
            'memoryUsage_stock',
            'applicationTime_trading',
            'databaseTime_trading',
            'numberOfSellOffers_trading',
            'numberOfBuyOffers_trading',
            # 'cpuUsage_traffic',
            # 'memoryUsage_traffic',
            # 'replicaId'
            ]
df_features = df[features]

# Kodowanie pudełek
le = LabelEncoder()
df['test'] = le.fit_transform(df['test'])

# Kopiowanie danych
df_encoded = df_features.copy()
df_encoded['test'] = df['test']

In [5]:
# Stwórz puste listy do przechowywania danych treningowych i testowych
X_train = []
X_test = []
y_train = []
y_test = []

In [6]:
def create_windows(X, y, window_size, step_size):
    X_windows = []
    y_windows = []

    # Przesuń okno po danych
    for i in range(0, len(X) - window_size, step_size):
        # Utwórz okno danych
        X_window = X.iloc[i:i + window_size]
        # Utwórz etykietę dla okna (etykieta ostatniej obserwacji w oknie)
        y_window = y.iloc[i + window_size]

        X_windows.append(X_window.values)
        y_windows.append(y_window)

    return np.array(X_windows), np.array(y_windows)

In [7]:
window_size = 500
step_size = 100
# Dla każdego unikalnego pudełka
for box in df_encoded['test'].unique():
    # Wybierz tylko rekordy dla tego pudełka
    box_data = df_encoded[df_encoded['test'] == box]
    
    # Oblicz punkt podziału (80% danych)
    split_point = int(len(box_data) * 0.8)
    
    # Dodaj pierwsze 80% rekordów do danych treningowych
    X_train_box = box_data.drop('test', axis=1).iloc[:split_point]
    y_train_box = box_data['test'].iloc[:split_point]
    
    # Dodaj ostatnie 20% rekordów do danych testowych
    X_test_box = box_data.drop('test', axis=1).iloc[split_point:]
    y_test_box = box_data['test'].iloc[split_point:]
    
    # Stwórz okienka dla danych treningowych
    X_train_windows, y_train_windows = create_windows(X_train_box, y_train_box, window_size, step_size)
    
    # Stwórz okienka dla danych testowych
    X_test_windows, y_test_windows = create_windows(X_test_box, y_test_box, window_size, step_size)
    
    # Dodaj okienka do list
    X_train.append(X_train_windows)
    y_train.append(y_train_windows)
    X_test.append(X_test_windows)
    y_test.append(y_test_windows)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.utils import to_categorical

In [9]:
# Połączenie danych treningowych dla wszystkich pudełek
X_train_combined = np.concatenate(X_train, axis=0)
y_train_combined = np.concatenate(y_train, axis=0)

In [10]:
# Zakodowanie etykiet w formacie one-hot encoding
y_train_combined_encoded = to_categorical(y_train_combined, num_classes=len(df['test'].unique()))

In [11]:
# Trenowanie modelu na połączonym zestawie danych
model = Sequential()
model.add(LSTM(100, input_shape=(window_size, X_train_combined.shape[2])))
model.add(Dense(len(df['test'].unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_combined, y_train_combined_encoded, epochs=10, batch_size=32)

Epoch 1/10
2024/2024 [==============================] - 381s 187ms/step - loss: 2.4253 - accuracy: 0.2155
Epoch 2/10
2024/2024 [==============================] - 396s 196ms/step - loss: 2.2988 - accuracy: 0.2360
Epoch 3/10
2024/2024 [==============================] - 368s 182ms/step - loss: 2.2494 - accuracy: 0.2357
Epoch 4/10
2024/2024 [==============================] - 345s 170ms/step - loss: 2.3192 - accuracy: 0.2224
Epoch 5/10
2024/2024 [==============================] - 347s 171ms/step - loss: 2.2574 - accuracy: 0.2283
Epoch 6/10
2024/2024 [==============================] - 347s 172ms/step - loss: 2.3066 - accuracy: 0.2204
Epoch 7/10
2024/2024 [==============================] - 350s 173ms/step - loss: 2.4457 - accuracy: 0.1951
Epoch 8/10
2024/2024 [==============================] - 422s 208ms/step - loss: 2.4212 - accuracy: 0.2028
Epoch 9/10
2024/2024 [==============================] - 416s 205ms/step - loss: 2.3805 - accuracy: 0.2119
Epoch 10/10
2024/2024 [=======================

In [13]:
for i in range(len(X_test)):
    y_test_encoded = to_categorical(y_test[i], num_classes=len(df['test'].unique()))
    loss, accuracy = model.evaluate(X_test[i], y_test_encoded)
    print(f"Test {i+1}: Loss = {loss}, Accuracy = {accuracy}")

34/34 [==============================] - 2s 49ms/step - loss: 3.0151 - accuracy: 0.0000e+00
Test 1: Loss = 3.0151193141937256, Accuracy = 0.0
34/34 [==============================] - 2s 49ms/step - loss: 2.9214 - accuracy: 0.0000e+00
Test 2: Loss = 2.921444892883301, Accuracy = 0.0
31/31 [==============================] - 2s 48ms/step - loss: 2.8317 - accuracy: 0.0000e+00
Test 3: Loss = 2.831681251525879, Accuracy = 0.0
33/33 [==============================] - 2s 48ms/step - loss: 2.6977 - accuracy: 0.0000e+00
Test 4: Loss = 2.6976773738861084, Accuracy = 0.0
26/26 [==============================] - 1s 47ms/step - loss: 3.0119 - accuracy: 0.0000e+00
Test 5: Loss = 3.0119175910949707, Accuracy = 0.0
30/30 [==============================] - 1s 47ms/step - loss: 2.4827 - accuracy: 0.7061
Test 6: Loss = 2.482677459716797, Accuracy = 0.706135630607605
16/16 [==============================] - 1s 46ms/step - loss: 3.0121 - accuracy: 0.0000e+00
Test 7: Loss = 3.012112855911255, Accuracy = 0.0
